## Imports

In [1]:
import pandas as pd
import os
from openpyxl import Workbook
from simplified_scrapy import SimplifiedDoc, utils
import matplotlib.pyplot as plt
from collections import defaultdict
pd.set_option('display.float_format', '{:.2f}'.format)

## Morningstar dataset conversion to Excel file (.xlsx)

In [2]:
"""
import os
files = []
for i in os.walk('./datasets'):
    files= i[2]
    
print(files)
"""

"\nimport os\nfiles = []\nfor i in os.walk('./datasets'):\n    files= i[2]\n    \nprint(files)\n"

In [3]:
"""
def readFile(filename):
    xml = utils.getFileContent(filename)
    doc = SimplifiedDoc(xml)
    tables = doc.selects('Worksheet').selects('Row').selects('Cell').text # Get all data
    sheetNames = doc.selects('Worksheet>ss:Name()') # Get sheet name
    return sheetNames,tables,filename.split('/')[2].split('.')[0]

def to_Excel(sheetNames,tables, filename):
    wb = Workbook() # Create Workbook
    for i in range(len(sheetNames)):
        worksheet = wb.create_sheet(sheetNames[i]) # Create sheet
        for row in tables[i]:
            worksheet.append(row)
    print('./datasets_excel/'+ filename + '.xlsx')
    wb.save('./datasets_excel/'+ filename + '.xlsx') # Save file
"""

"\ndef readFile(filename):\n    xml = utils.getFileContent(filename)\n    doc = SimplifiedDoc(xml)\n    tables = doc.selects('Worksheet').selects('Row').selects('Cell').text # Get all data\n    sheetNames = doc.selects('Worksheet>ss:Name()') # Get sheet name\n    return sheetNames,tables,filename.split('/')[2].split('.')[0]\n\ndef to_Excel(sheetNames,tables, filename):\n    wb = Workbook() # Create Workbook\n    for i in range(len(sheetNames)):\n        worksheet = wb.create_sheet(sheetNames[i]) # Create sheet\n        for row in tables[i]:\n            worksheet.append(row)\n    print('./datasets_excel/'+ filename + '.xlsx')\n    wb.save('./datasets_excel/'+ filename + '.xlsx') # Save file\n"

In [4]:
"""
for filename in (os.listdir("./datasets")):
        name = filename.split('.')[0]
        print(name)
        
        dst = name + ".xml"
        src ='./datasets/'+ filename
        dst ='./datasets/'+ dst
          
        # rename() function will
        # rename all the files
        os.rename(src, dst)
"""

'\nfor filename in (os.listdir("./datasets")):\n        name = filename.split(\'.\')[0]\n        print(name)\n        \n        dst = name + ".xml"\n        src =\'./datasets/\'+ filename\n        dst =\'./datasets/\'+ dst\n          \n        # rename() function will\n        # rename all the files\n        os.rename(src, dst)\n'

In [5]:
"""
for file in files:
    to_Excel(*readFile("./datasets/" + file))
"""

'\nfor file in files:\n    to_Excel(*readFile("./datasets/" + file))\n'

## Import datasets into Pandas dataframes

In [6]:
files = []
for i in os.walk('./datasets_excel/'):
    files=i[2]

workbooks = {}
for file in files:
    workbooks[file.split('.')[0]] = pd.read_excel('./datasets_excel/' + file, sheet_name=None)

In [7]:
print(list(workbooks.keys()))

['ADH_FY', 'ADH_HY', 'ANZ_FY', 'ANZ_HY', 'BCI_FY', 'BCI_HY', 'BFG_FY', 'BFG_HY', 'BHP_FY', 'BHP_HY', 'BNL_FY', 'BNL_HY', 'BPG_FY', 'BPG_HY', 'BTR_FY', 'BTR_HY', 'CBA_FY', 'CBA_HY', 'CCC_FY', 'CCC_HY', 'CIM_FY', 'CIM_HY', 'CVR_FY', 'CVR_HY', 'DGX_FY', 'DGX_HY', 'DTZ_FY', 'DTZ_HY', 'GMN_FY', 'GMN_HY', 'HWK_FY', 'HWK_HY', 'JBH_FY', 'JBH_HY', 'LCT_FY', 'LCT_HY', 'MEB_FY', 'MEB_HY', 'MFG_FY', 'MFG_HY', 'MGC_FY', 'MGC_HY', 'MQG_FY', 'MQG_HY', 'NAB_FY', 'NAB_HY', 'NCM_FY', 'NCM_HY', 'NGI_FY', 'NGI_HY', 'REX_FY', 'REX_HY', 'RIO_FY', 'RIO_HY', 'RNO_FY', 'RNO_HY', 'SRX_FY', 'SRX_HY', 'SRZ_FY', 'SRZ_HY', 'WBC_FY', 'WBC_HY', 'WCC_FY', 'WCC_HY']


## Cleaning

### Normalising statement items

In [8]:
statement_items = defaultdict(set)

for workbook in workbooks:
    for sheet in workbooks[workbook]:
        if sheet != 'Sheet' and 'Item' in workbooks[workbook][sheet]:
            statement_items[sheet].update(workbooks[workbook][sheet]['Item'])
            
statement_items.keys()

dict_keys(['Profit Loss', 'Balance Sheet', 'Cash Flow', 'Per Share Statisticts', 'Sundry Analysis', 'Growth Rates', 'Ratio Analysis', 'Asset Base Analysis'])

### Combining HY and FY datasets

In [50]:
workbooks_combined = defaultdict(dict)
workbooks_keys = list(workbooks.keys())

for i in range(len(workbooks)//2):
    for sheet in statement_items:
        workbook_FY = workbooks[workbooks_keys[i*2]]
        workbook_HY = workbooks[workbooks_keys[i*2 + 1]]
        new_sheet = _ 
        
        if sheet in workbook_FY:
            sheet_FY = workbook_FY[sheet].drop(['ASX Code', 'Company Name'], axis=1)
            sheet_HY = workbook_HY[sheet].drop(['ASX Code', 'Company Name'], axis=1)

            new_sheet = pd.concat([sheet_FY, sheet_HY], axis=1, join='inner')
            new_sheet = new_sheet.loc[:,~new_sheet.columns.duplicated()]
            new_sheet = new_sheet.set_index(['Item'])
            
            new_sheet.columns = pd.to_datetime(new_sheet.columns, format='%m/%y')
            new_sheet = new_sheet.sort_index(axis=1)
            
            new_sheet = new_sheet.replace('--', 0, regex=True)
            new_sheet.astype('float')

            workbooks_combined[workbooks_keys[i*2].split('_')[0]][sheet] = new_sheet.transpose()    
        else:
            print("Missing tab:", workbooks_keys[i*2], sheet)            

Missing tab: ANZ_FY Sundry Analysis
Missing tab: CBA_FY Sundry Analysis
Missing tab: NAB_FY Sundry Analysis
Missing tab: WBC_FY Sundry Analysis


In [46]:
workbooks_combined.keys()

dict_keys(['ADH', 'ANZ', 'BCI', 'BFG', 'BHP', 'BNL', 'BPG', 'BTR', 'CBA', 'CCC', 'CIM', 'CVR', 'DGX', 'DTZ', 'GMN', 'HWK', 'JBH', 'LCT', 'MEB', 'MFG', 'MGC', 'MQG', 'NAB', 'NCM', 'NGI', 'REX', 'RIO', 'RNO', 'SRX', 'SRZ', 'WBC', 'WCC'])

In [49]:
workbooks_combined['ADH']['Profit Loss']

Item,Operating Revenue,Other Revenue,Total Revenue Excluding Interest,Operating Expenses,EBITDA,Depreciation,Amortisation,Depreciation and Amortisation,EBIT,Interest Revenue,...,Net Profit after Tax Before Abnormals,Abnormals,Abnormals Tax,Net Abnormals,Reported NPAT After Abnormals,Outside Equity Interests,Shares Outstanding at Period End,Weighted Average Number of Shares,EPS Adjusted (cents/share),EPS After Abnormals (cents/share)
2014-12-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,165874785,0,0.00,0.00
2015-06-01,210878000.00,166000.00,211044000.00,-188019000.00,23025000.00,-3858000.00,-728000.00,-4586000.00,18439000.00,445000.00,...,2942000.00,0.00,0.00,0.00,745000.00,0.00,165875000.00,151471000.00,1.94,0.49
2015-12-01,117821000.00,125000.00,117946000.00,-95441000.00,22505000.00,-2593000.00,0.00,-2593000.00,19912000.00,24000.00,...,13254000.00,0.00,0.00,0.00,13254000.00,0.00,165874785.00,165875000.00,7.99,7.99
2016-06-01,253182000.00,406000.00,253588000.00,-207574000.00,46014000.00,-4649000.00,-617000.00,-5266000.00,40748000.00,81000.00,...,27172000.00,0.00,0.00,0.00,27172000.00,0.00,165875000.00,165875000.00,16.38,16.38
2016-12-01,124548000.00,37000.00,124585000.00,-109832000.00,14753000.00,-2751000.00,0.00,-2751000.00,12002000.00,46000.00,...,8581000.00,0.00,0.00,0.00,8581000.00,0.00,165874785.00,165875000.00,5.17,5.17
2017-06-01,264964000.00,190000.00,265154000.00,-228465000.00,36689000.00,-5164000.00,-713000.00,-5877000.00,30812000.00,55000.00,...,21017000.00,0.00,0.00,0.00,21017000.00,0.00,165875000.00,165875000.00,12.67,12.67
2017-12-01,149047000.00,61000.00,149108000.00,-124929000.00,24179000.00,-3300000.00,0.00,-3300000.00,20879000.00,19000.00,...,13940000.00,0.00,0.00,0.00,13940000.00,0.00,165874785.00,165875000.00,8.40,8.40
2018-06-01,314769000.00,0.00,314769000.00,-262677000.00,52092000.00,-5719000.00,-1112000.00,-6831000.00,45261000.00,44000.00,...,30561000.00,0.00,0.00,0.00,30561000.00,0.00,165875000.00,165875000.00,18.30,18.30
2018-12-01,164377000.00,225000.00,164602000.00,-138704000.00,25898000.00,-3752000.00,0.00,-3752000.00,22146000.00,35000.00,...,15108000.00,-215000.00,0.00,-215000.00,14893000.00,0.00,165874785.00,165875000.00,8.93,8.93
2019-06-01,344430000.00,270000.00,344700000.00,-293565000.00,51135000.00,-6394000.00,-1295000.00,-7689000.00,43446000.00,77000.00,...,29643000.00,0.00,0.00,0.00,29643000.00,0.00,165875000.00,165875000.00,17.87,17.87


## Visualisation